In [ ]:
!pip freeze | grep scikit-learn

In [1]:
import pickle
import pandas as pd

In [7]:
with open('modelCopy1.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [8]:
categorical = ['PULocationID', 'DOLocationID']
year = 2021
month = 2

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [11]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet')
#df = read_data(f'../../data/yellow/yellow_tripdata_2022-03.parquet')

In [12]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
y_pred.std()

5.556602554785001

In [7]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [8]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [10]:
df_result.to_parquet(
    'score_out',
    engine='pyarrow',
    compression=None,
    index=False
)

In [9]:
df_result.head()

,ride_id,predicted_duration
0,2021/02_0,18.527783
1,2021/02_1,23.065782
2,2021/02_2,33.686359
3,2021/02_3,23.757436
4,2021/02_4,21.492904
